In [1]:
import re
import numpy as np
import pandas as pd
from joblib import dump, load
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [2]:
from text_cleaning import clean_up

In [3]:
categories = [
    'faciales',
    'capilares',
    'corporales',
    'maquillaje',
    'fragancias',
    'higiene'
]

In [4]:
dictionary = {
    each_category:pd.read_json(
        f'./data/{each_category}.json', orient='records'
    ) for each_category in categories
}

for each_category in categories:
    dictionary[each_category]['product_category'] = each_category

dictionary['faciales'].head()

,product_name,product_chunk,product_image,product_description,product_more_info,product_category
0,Contorno De Ojos En Gel Hidratante,Código: 00007\n15 ml\nA $200,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales
1,Crema Facial Exfoliante Aclarante,Código: 01176\n125 ml\nA $290,https://www.yvesrocher.com.mx/Areas/YR/images/...,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales
2,Maxi Gel Facial Limpiador Hidratante,Código: 01340\n390 ml\nA $340,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales
3,Mascarilla Facial Nocturna Aclarante,Código: 06043\n75 ml\nA $359,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales
4,Crema Facial Suavizante Día/noche,Código: 11783\n50 ml\nA $175,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",faciales


In [5]:
df = pd.concat(dictionary.values())
df.head()

,product_name,product_chunk,product_image,product_description,product_more_info,product_category
0,Contorno De Ojos En Gel Hidratante,Código: 00007\n15 ml\nA $200,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales
1,Crema Facial Exfoliante Aclarante,Código: 01176\n125 ml\nA $290,https://www.yvesrocher.com.mx/Areas/YR/images/...,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales
2,Maxi Gel Facial Limpiador Hidratante,Código: 01340\n390 ml\nA $340,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales
3,Mascarilla Facial Nocturna Aclarante,Código: 06043\n75 ml\nA $359,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales
4,Crema Facial Suavizante Día/noche,Código: 11783\n50 ml\nA $175,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",faciales


In [6]:
df.isnull().sum(axis=0)

product_name           0
product_chunk          0
product_image          0
product_description    0
product_more_info      0
product_category       0
dtype: int64

In [7]:
df.product_category.value_counts()

maquillaje    290
higiene       119
faciales       99
corporales     64
fragancias     50
capilares      33
Name: product_category, dtype: int64

In [8]:
df.product_chunk.str.split(expand=True).head()

,0,1,2,3,4,5
0,Código:,00007,15,ml,A,$200
1,Código:,01176,125,ml,A,$290
2,Código:,01340,390,ml,A,$340
3,Código:,06043,75,ml,A,$359
4,Código:,11783,50,ml,A,$175


In [9]:
df[['product_code', 'product_price']] = df.product_chunk.str.split(expand=True)[[1,5]]

In [10]:
df.head()

,product_name,product_chunk,product_image,product_description,product_more_info,product_category,product_code,product_price
0,Contorno De Ojos En Gel Hidratante,Código: 00007\n15 ml\nA $200,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales,00007,$200
1,Crema Facial Exfoliante Aclarante,Código: 01176\n125 ml\nA $290,https://www.yvesrocher.com.mx/Areas/YR/images/...,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales,01176,$290
2,Maxi Gel Facial Limpiador Hidratante,Código: 01340\n390 ml\nA $340,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales,01340,$340
3,Mascarilla Facial Nocturna Aclarante,Código: 06043\n75 ml\nA $359,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales,06043,$359
4,Crema Facial Suavizante Día/noche,Código: 11783\n50 ml\nA $175,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",faciales,11783,$175


In [11]:
df['product_price'] = df.product_price.str.replace(r'\$', '').astype(int)

In [12]:
df.drop(columns=['product_chunk'], inplace=True)

In [13]:
df.head()

,product_name,product_image,product_description,product_more_info,product_category,product_code,product_price
0,Contorno De Ojos En Gel Hidratante,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales,00007,200
1,Crema Facial Exfoliante Aclarante,https://www.yvesrocher.com.mx/Areas/YR/images/...,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales,01176,290
2,Maxi Gel Facial Limpiador Hidratante,https://www.yvesrocher.com.mx/Areas/YR/images/...,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,faciales,01340,340
3,Mascarilla Facial Nocturna Aclarante,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,faciales,06043,359
4,Crema Facial Suavizante Día/noche,https://www.yvesrocher.com.mx/Areas/YR/images/...,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",faciales,11783,175


In [14]:
df.drop_duplicates(subset='product_code', inplace=True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 0 to 118
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_name         621 non-null    object
 1   product_image        621 non-null    object
 2   product_description  621 non-null    object
 3   product_more_info    621 non-null    object
 4   product_category     621 non-null    object
 5   product_code         621 non-null    object
 6   product_price        621 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 38.8+ KB


In [16]:
df = df[[
    'product_code', 'product_name', 'product_price',
    'product_category', 'product_description',
    'product_more_info', 'product_image',
    
]]
df.head()

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image
0,00007,Contorno De Ojos En Gel Hidratante,200,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...
1,01176,Crema Facial Exfoliante Aclarante,290,faciales,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...
2,01340,Maxi Gel Facial Limpiador Hidratante,340,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...
3,06043,Mascarilla Facial Nocturna Aclarante,359,faciales,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...
4,11783,Crema Facial Suavizante Día/noche,175,faciales,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",https://www.yvesrocher.com.mx/Areas/YR/images/...


In [17]:
df.to_json('./data/products.json', orient='records', indent=2)

In [18]:
# create a column with all words from text columns
text_cols = ['product_name', 'product_category', 'product_description', 'product_more_info']

text_series = df[text_cols].apply(lambda x: " ".join(x), axis=1).apply(clean_up)
text_series.head()

0    contorn ojos gel hidrat facial expert cosmet v...
1    crem facial exfoli aclar facial exfoli suav el...
2    maxi gel facial limpiador hidrat facial expert...
3    mascarill facial nocturn aclar facial piel sua...
4    crem facial suaviz dia noch facial hidrat suav...
dtype: object

In [19]:
dump(text_series, './data/text_series.joblib')

['text_series.joblib']